# Introduction to the bioinformatics armory

## Question ini

Count nucleotide acids in a DNA sequence with *Bio.Seq* from Biopython

In [6]:
from Bio.Seq import Seq

def countNt(str):
    mySeq = Seq(str)
    res = [mySeq.count('A'), mySeq.count('C'), mySeq.count('G'), mySeq.count('T')]
    return(res)

def printCountNt(s):
    res = countNt(s)
    print(' '.join(list(map(str, res))))
    
myStr = "AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGC"

countNt(myStr)
printCountNt(myStr)

20 12 17 21


In [7]:
with open('rosalind_ini.txt', 'r') as f:
    line = f.readline()
    
printCountNt(line)

225 231 220 239
